
This notebook tutorial is intended to cover a variety of coding themes to empower new GBDX Notebook developers.  

Rather than presenting an end-to-end application on a particular topic, I'll present a _veritable cornucopia_ of functional topics, with linked  examples of them in application -- a _potpourri_ of libraries and coding tricks, a _menagerie_ of syntax I use frequently ... but, I digress.

## Topic 1:  Changing between different forms of vector geometries and useful visualizations thereof

As a relatively new python coder on GBDX, I've found it useful, if not necessary, to be able to move between four fundamental forms of vector geometries: 

1. <a href="https://en.wikipedia.org/wiki/Shapefile" target="_blank">ESRI Shapefiles</a> represent industry standard and are the de facto format for many users/customers
2. <a href="https://en.wikipedia.org/wiki/Well-known_text" target="_blank">Well-Known Test (WKT)</a> is one of the easiest forms of geometry to create (more below).  It is used when making geographic queries of GBDX Vector Services for imagery and other content
3. <a href="https://shapely.readthedocs.io/en/stable/manual.html" target="_blank">Shapely</a> geometries enable numerous geometric tests such as `intersection` and `contains`, as well as the `transform` operator for coordinate system projections
4. <a href="http://geojson.org/" target="_blank">GeoJSON</a> is used when interacting with GBDX's default slippy map, or other interesting visualizations like Folium (<a href="https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis" target="_blank">example</a>)


#### ESRI Shapefiles

ESRI shapefiles represent industry standard and are the de facto format for many users/customers.  To start working with one in GBDX Notebooks, ensure your ESRI shapefile is uploaded to your GBDX Notebooks file tree.  At a minimum, your working folder should contain corresponding `shp`, `shx`, and `dbf` files for the shapefile you wish to work with.

_Note: The data used in this part of the notebook was provided by Ecopia. The data is stored in an s3 bucket. The bucket access has been hidden, however, you can still access the data by using the `nbfirerisk` <a href="https://github.com/GeoBigData/nbfirerisk" target="_blank" rel="noopener">github package</a>.  If you ever want to use your own shapefile data, simply upload the files into the Jupyter File tree._


In [ ]:
#imports
# for shapefile operations
try:
    import fiona
except:
    print("Installing fiona...")
    !pip install fiona -q
    import fiona

# for Jupyter file tree operations
import os   

#to retrieve shapefile data
try:
    import nbfirerisk
except:
    print("Installing nbfirerisk...")
    !pip install git+https://github.com/GeoBigData/nbfirerisk.git -qq 
    import nbfirerisk
    
try:
    import rasterio
except:
    print("Installing rasterio...")
    !pip install rasterio -q 
    import rasterio    

In [ ]:
shp_files = nbfirerisk.ecopia_buildings_lake_keswick #retrieve shapefiles
#shapefile = fiona.open(os.environ['HOME']+'/your_file.shp') for opening data stored in your file tree
shapefile = fiona.open(shp_files) #for opening the example files
print('Number of geometries in shapefile:', len(shapefile))

#### Well-Known Text (WKT): for _GBDX Queries_

If I have a workflow that requires me to create geometries, I typically won't start with a Shapefile, simply because I find them cumbersome to generate.  WKT is an extremely simple alternative (text string) and has a unique role in GBDX -- namely, __WKT is used to query GBDX <a href="https://gbdxdocs.digitalglobe.com/docs/vector-services-overview" target="_blank">Vector Services</a>__ for imagery and other platform content. 

<a href="https://arthur-e.github.io/Wicket/sandbox-gmaps3.html" target="_blank">Wicket</a> is a great online resource for 
creating simple WKT strings via an interactive map.  I'll often use Wicket in combination with this 
<a href="https://discover.digitalglobe.com/" target="_blank">DigitalGlobe Search and Discovery</a> portal to refine a workflow's area of interest.

#### Here are two examples of querying Vector Services for imagery.  

__Note:__ the second query demonstrates use of the `now` parameter, which is really useful when searching for current imagery, 

In [ ]:
# QUERY 1

# define an Area of Interest (AOI) using a wkt string (which I pulled from Wicket)
aoi_wkt_string = 'POLYGON((-122.44212989281158 40.6204217694745,-122.4259508080338 40.6204217694745,-122.4259508080338 40.6135,-122.44212989281158 40.6135,-122.44212989281158 40.6204217694745))'

# Search for GBDX imagery that intersects with the AOI

filters = ["sensorPlatformName = 'WORLDVIEW03_VNIR','WORLDVIEW03_SWIR', 'WORLDVIEW02' "] #, "cloudCover < 15"]

from gbdxtools import CatalogImage, Interface
gbdx = Interface()

imagery_results = gbdx.catalog.search(searchAreaWkt=aoi_wkt_string, startDate="2018-01-01T00:00:00.000Z", endDate="2018-03-31T00:00:00.000Z", filters=filters)
# Search for OSM street data that intersects with the AOI 

print()
print('Returns for Q1 CY2018 WV-2 and WV-3 imagery over AOI:')
for r in imagery_results:
    props = r['properties']
    print('      ',props['catalogID'], props['timestamp'][0:10], props['sensorPlatformName'])

# QUERY 2
    
max_cloud_cover = 25 # upper bound for acceptable cloud cover percentage

query = "(item_type:WV03_VNIR OR item_type:WV03_SWIR OR item_type:WV02)"
query += " AND NOT item_type:IDAHOImage AND item_type:DigitalGlobeAcquisition" 
query += ' AND attributes.cloudCover_int:<'+str(max_cloud_cover)
query += " AND item_date:>=now-7d" # only return imagery less than 1 week old
#query += " AND item_date:>=now-48h" # only return imagery less than 48 hours old

imagery_results = gbdx.vectors.query(aoi_wkt_string, query, count=100) # count establishes an upper limit for the return

print()
print('Additional filters applied: less than one week old, less than ' + str(max_cloud_cover) + '% cloud cover')
for r in imagery_results:
    props = r['properties']
    print('      ',props['id'], props['item_date'][0:10], props['attributes']['sensorPlatformName'])



By the way, if you find an image you'd like to use via the above call, the following syntax can be useful to determine if it's pre-staged or needs to be ordered.

In [ ]:
catalog_id = '104001003E7F4B00'

try:
    strip = CatalogImage(catalog_id)
    print()
    print('Strip geographic bounds:', strip.bounds)
    print('Strip raster dimensions:', strip.shape)

except:
    print()
    order_id = gbdx.ordering.order(catalog_id)
    print('Image ordered for GBDX staging:  Order ID', order_id)
    print('Typical factory delivery times range from 6-48 hours.')

#### Shapely geometries:  for _Geo-processing_

If I need to perform GIS processing operations, I usually convert the data to <a href="https://shapely.readthedocs.io/en/stable/manual.html" target="_blank" rel="noopener">Shapely</a> geometries.  This library enables numerous geometric tests such as `intersection` and `contains`, as well as the handy operators like `buffer` and `transform` for coordinate system projections. 

This next snippet demonstrates converting a shapefile into Shapely geometries, then filtering the polygons based on a polygon area of interest (AOI).

In [ ]:
%%time
# Shapely for processing geometries
import shapely
from shapely.geometry import shape


# convert the AOI from wkt to a shapely geometry 
shapely_aoi = shapely.wkt.loads(aoi_wkt_string)

shapely_building_polygons_within_aoi = []

# convert to shapely, re-project, and then filter the polygons within the aoi using the "contains" shapely test operator
for i, shapes in enumerate(shapefile):
    polygon = shape(shapefile[i]['geometry'])
    
    if shapely_aoi.contains(polygon):
        shapely_building_polygons_within_aoi.append(polygon)
        
num_building_polygons_within_aoi = len(shapely_building_polygons_within_aoi)

print()
print('Original shapefile contained', len(shapefile), 'polygons.')

print()
print(num_building_polygons_within_aoi, 'polygons contained within AOI.')
        


#### GeoJSON:  for _Visualizations_

Finally, <a href="http://geojson.org/" target="_blank" rel="noopener">GeoJSON</a> is used when interacting with GBDX's default slippy map, or other interesting visualizations like Folium (<a href="https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis" target="_blank" rel="noopener">example</a>)

The obvious advantage of GBDX's slippy map is its base imagery.  _Important:_ In order for geometry to render properly on a GBDX slippy map, it must be in WGS84 coordinates.

Here I'll demonstrate how to convert from WKT or Shapely geometry to GeoJSON.

In [ ]:
from shapely.wkt import loads as from_wkt
from shapely.geometry import mapping as to_geojson

# convert wkt to geojson and display on a GBDX slippy map with DG base imagery

aoi_from_wkt_to_geojson = to_geojson(from_wkt(aoi_wkt_string))

Conversely, converting from Shapely geometry to GeoJSON is also easy.

In [ ]:
aoi_from_shapely_to_geojson = shapely_aoi.__geo_interface__

Folium is a tool for visualizing features on a slippy map. It has lots of cool rendering options (<a href="https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis" target="_blank" rel="noopener">Example 1</a>, <a href="https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium" target="_blank" rel="noopener" >Example 2</a>, 
<a href="http://qingkaikong.blogspot.com/2016/06/using-folium-3-heatmap.html" target="_blank" rel="noopener">Example 3</a>
)


In [ ]:
try:
    import folium
except:
    ! pip install folium -q
    import folium

aoi_centroid = shapely_aoi.centroid.coords[0]
    
m = folium.Map([aoi_centroid[1], aoi_centroid[0]], zoom_start=15)

# add the AOI to the map
folium.GeoJson(aoi_from_shapely_to_geojson).add_to(m)

m

## Topic 2:  Changing between geographic (lon, lat) and raster (pixel $x,y$) coordinate systems

GBDX images have affine transform properties _in their metadata (!)_ that make raster operations with vector geometry easy. This metadata is a very important distinction between a GBDX catalog image and simple 2D numpy array; however, the latter may be derived via `.read()`(shown below). 

In [ ]:
# for image visualization and plotting
import matplotlib.pyplot as plt
# for converting shapely geometry into a raster mask
import rasterio
import rasterio.features

# for numerical array operations
import numpy as np

# first let's load an image corresponding to the bounding box above

aoi_bounds = shapely_aoi.bounds

bbox = [aoi_bounds[0], aoi_bounds[1], aoi_bounds[2], aoi_bounds[3]]

image = CatalogImage(catalog_id, bbox=bbox)
nrows = image.shape[1]
ncols = image.shape[2]

print()
print(type(image))

simple_array = image.read()

print()
print(type(simple_array))


plt.figure(figsize=(20, 20))
plt.imshow(image.rgb())
plt.title('Image Excerpt from ' + str(catalog_id))
plt.axis('off')
plt.show()

Now let's turn our attention slightly to the building footprint geometries we harvested from the ESRI shapefile above.  Btw, these <a href="http://explore.digitalglobe.com/GBDX-Building-Footprints.html" target="_blank" rel="noopener">building footprints</a> were created by Ecopia using GBDX!




In [ ]:
raster_buildings = rasterio.features.rasterize(
    shapely_building_polygons_within_aoi, # list of shapely polygons
    out_shape = (nrows, ncols),
    transform= image.affine,
    fill=0,
    all_touched=True,
    dtype=np.uint8)

plt.figure(figsize=(20, 20))
plt.imshow(raster_buildings)
plt.title('Building Footprints Raster Mask')
plt.axis('off')
plt.show()



#### <a href="http://scikit-image.org/docs/dev/api/skimage.html" target="_blank" rel="noopener">Scikit-image</a> is a wonderfully useful library, in general.  I particularly like their raster region analysis utilities.  

With a binary mask in hand, you can readily extract $(x,y)$ coordinates for each region and perform for targeted spectral analysis or visualizations.  Here I'll simply demonstrate how to differentiate between bright and dark roof colors, but there is so much more you can do with this type of technique. 

In [ ]:
%%time

from skimage.measure import label, regionprops

# for math operations
import numpy as np

label_img = label(raster_buildings)

regions = regionprops(label_img)

rgb = image.rgb()

overlay = np.empty((nrows,ncols,3)).astype('uint8')

overlay[:] = rgb

bright_count = 0

dark_count = 0


for i, props in enumerate(regions):
    
    pixel_count = props.area
    
    building_raster_coords = props.coords
    
    mag_array = []
    
    # calculate the mean rgb magnitude for a single building footprint
    
    for j in range(pixel_count):
        
        pixel = rgb[building_raster_coords[j,0], building_raster_coords[j,1],:] 
        
        pixel_norm = np.linalg.norm(pixel)
        
        mag_array.append(pixel_norm)  
        
    mean_mag = np.mean(mag_array)
    
    #print(mean_mag)
  
    if mean_mag > 300:
        overlay[building_raster_coords[:,0], building_raster_coords[:,1], 1:2] = 0
        overlay[building_raster_coords[:,0], building_raster_coords[:,1], 0] = np.max(rgb)
              
        bright_count = bright_count + 1
        
    else:    
        overlay[building_raster_coords[:,0], building_raster_coords[:,1], 0:1] = 0
        overlay[building_raster_coords[:,0], building_raster_coords[:,1], 2] = np.max(rgb)
               
        dark_count = dark_count + 1

fig = plt.figure(figsize=(20, 15))

plt.subplot(2,1,1)

plt.imshow(rgb)
plt.title('Image Excerpt from ' + str(catalog_id))
plt.axis('off')


plt.subplot(2,1,2)
plt.imshow(overlay)
plt.title('Bright and Dark Buildings')
plt.axis('off')

ax = fig.add_subplot(2, 1, 2)
plt.plot([], 'bo', 
        label='Dark')
plt.plot([], 'mo', 
        label='Bright')
ax.legend()

plt.tight_layout()
plt.show()

print(bright_count, 'and', dark_count, 'bright and dark roofs counted, respectively.')
print()
        

#### GBDX also has some useful built-in methods to quickly transform between pixel and geographic coordinate systems.

If I were interested in retrieving the geocoords for a particular pixel, I can get them via the following simple call.

In [ ]:
# assume I'm interested in finding geocoords for the pixel with raster coordinates of [44,22]

geo_coords = image.__geo_transform__.fwd(44,22)

geo_coords

Conversely, I can also determine the pixel coordinates for a geographic point.

In [ ]:
pixel_coords = image.__geo_transform__.rev(-122.44157393848006, 40.62014447898311)

pixel_coords

Finally, in the scikit-image example above, I showed how to retreive comprehensive pixel coordinates for an abitrary vector polygon.  That may be overkill for many applications.  If you can get by with just the polygon pixel bounds `(min_x, min_y, max_x, max_y)`, the following built-in `pxbounds` method may be preferred due to its simplicity. 

In [ ]:
#grab the shape associated with a single building footprint

index = 100

shape = shapely_building_polygons_within_aoi[index]

pixel_bounds = image.pxbounds(shape)

print()
print('Pixel bounds for building footprint ' + str(index+1) + ':', pixel_bounds)

print()
print('Note:  the actual building footprint is not rectangular.')

shape

## Topic 3:  Co-registration and Histogram equalization as considerations for change detection 

Change detection, in its most basic implementation, involves differencing two images of the same area taken at different times.  It is reasonable that any pixels that have not changed would have values close to zero, while deviation from that will be due to brightness variations.  In practice, a variety of factors can introduce unwanted additional variability, to include differences in dynamic range, shadow positions (solar geometry), and artifacts due to geolocation error.  The next section is not intended to be a lengthy primer on change detection, but simply to introduce co-registration and histogram equalization as possible ways to mitigate such artifacts.

This example looks at an damage to an area of Mosul, Iraq.  The differences between 2016 and 2017 are readily evident. 

In [ ]:
image_bbox_wkt = 'POLYGON((43.13066461080268 36.34525376787812,43.146114134728464 36.34525376787812,43.146114134728464 36.333984236982346,43.13066461080268 36.333984236982346,43.13066461080268 36.34525376787812))'

aoi_bounds = shapely.wkt.loads(image_bbox_wkt).bounds

bbox = [aoi_bounds[0], aoi_bounds[1], aoi_bounds[2], aoi_bounds[3]]


catalog_id_2016 = '1040010025191900' 

catalog_id_2017 = '1040010032486E00'

image_2016 = CatalogImage(catalog_id_2016, bbox=bbox, acomp=True)

image_2017 = CatalogImage(catalog_id_2017, bbox=bbox, acomp=True)

plt.figure(figsize=(20, 15))

plt.subplot(1,2,1)
plt.imshow(image_2016.rgb())
plt.title('Mosul, Nov 2016')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(image_2017.rgb())
plt.title('Mosul, Aug 2017')
plt.axis('off')

plt.tight_layout()
plt.show()


It's obvious we have the same spatial area but the 2017 image is brighter, which makes comparison more difficult. You can think of this as looking at two photographs once which is underexposed and one that is overexposed - it is hard to compare what has changed because colors are so different.  We'll address this using a technique called histogram equalization below.

To simplify image-to-image differencing, we'll convert the color images to grayscale.  Let's see what happens if we just resample and difference the grayscale images.  

In [ ]:
# for custom colorbar plot layout
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

def basic_change_detection_plot(array, title_caption='', colorbar_status='off', colormap_choice='jet'):
    
    plt.figure(figsize=(20, 20))
    plt.title(title_caption)
    plt.axis('off')
    if colorbar_status == 'on':
        aspect=20
        pad_fraction = 0.5
        vmax = np.max(np.abs(array))
        vmin = -1*vmax # ensure symmetry in colorbar bounds, making 0
        ax = plt.gca()
        im = ax.imshow(array, cmap = colormap_choice, vmax=vmax,vmin=vmin)
        divider = make_axes_locatable(ax)
        width = axes_size.AxesY(ax, aspect=1./aspect)
        pad = axes_size.Fraction(pad_fraction, width)
        cax = divider.append_axes("right", size =width, pad=pad)
        ax.set_title(title_caption, loc='center')
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        plt.colorbar(im, cax=cax)

    else:
        plt.imshow(array)
    
    plt.show()  

def rgb_to_grayscale(any_rgb):
    gs = 0.299*any_rgb[:,:,0] + 0.587*any_rgb[:,:,1] + 0.114*any_rgb[:,:,2]
    return gs.astype('uint8')

gs_2016 = rgb_to_grayscale(image_2016.rgb())

gs_2017 = rgb_to_grayscale(image_2017.rgb())
 

# for resampling array sizes    
from skimage.transform import resize

resampled_gs_2017 = resize(gs_2017, gs_2016.shape, order=1, mode='constant', clip=True)

difference = resampled_gs_2017 - gs_2016
              

basic_change_detection_plot(difference, 
                            title_caption='Method 1:  Difference (2017-2016)', 
                            colorbar_status = 'on',
                            colormap_choice = 'RdBu'
                           )
            



Recall creating the difference image above involved resampling the 2017 image to the same array size as the 2016 image.  We're fortunate that the two source images have high spatial accuracy, in that limited edge artifacts from misregistration are visible.

Now, let's try co-registering the two images and applying histogram equalization.  

Beyond just simply resampling the image, co-registration corrects image misalignment by invoking an <a href="https://en.wikipedia.org/wiki/Affine_transformation" target="_blank" rel="noopener">affine transformation</a>.  The great thing about the co-registration code below is that it automatically generates common reference tie points across two 8-bit images.  It's pretty quick as long as the image pairs have relatively high spatial accuracy and shared edge features.

Once we've ensured quality image alignment, then histogram equalization may be invoked to create similar brightness levels across the images.  I chose to do this because of the noticeably different contrast levels in the two images, but it should be acknowledged that the histogram of the equalized 2017 image is artificial.

In [ ]:
# for co-registration
try:
    import cv2
except:
    !pip install "opencv-python" 
    import cv2

# for histogram matching
!pip install -q rio_hist
from rio_hist.match import histogram_match
    
# we're going to warp the 2017 image to the 2016 as an affine transformation    
before_chip= gs_2016
after_chip = gs_2017

# Find size of image1
sz = before_chip.shape

# Define the motion model
warp_mode = cv2.MOTION_AFFINE
#warp_mode = cv2.MOTION_HOMOGRAPHY

# Define 2x3 or 3x3 matrices and initialize the matrix to identity
if warp_mode == cv2.MOTION_HOMOGRAPHY :
    warp_matrix = np.eye(3, 3, dtype=np.float32)
else :
    warp_matrix = np.eye(2, 3, dtype=np.float32)

# Specify the number of iterations.
number_of_iterations = 50;

# Specify the threshold of the increment
# in the correlation coefficient between two iterations
termination_eps = 1e-10;

# Define termination criteria
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

# Run the ECC algorithm. The results are stored in warp_matrix.
(cc, warp_matrix) = cv2.findTransformECC (before_chip,after_chip,warp_matrix, warp_mode, criteria, inputMask=None, gaussFiltSize=1)

if warp_mode == cv2.MOTION_HOMOGRAPHY :
    # Use warpPerspective for Homography 
    after_chip_aligned = cv2.warpPerspective (after_chip, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
else :
    # Use warpAffine for Translation, Euclidean and Affine
    after_chip_aligned = cv2.warpAffine(after_chip, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

# # Show final results, if you wanna...
# plt.figure(figsize=(20,20))
# plt.subplot(131);plt.imshow(before_chip, cmap='gray');plt.title('Chip Before')
# plt.subplot(132);plt.imshow(after_chip, cmap='gray');plt.title('Chip After')
# plt.subplot(133);plt.imshow(after_chip_aligned, cmap='gray');plt.title('Chip After Aligned with Chip Before')

# plt.show

coregistered_gs_2017 = after_chip_aligned

# now apply histogram equalization

equalized_coregistered_gs_2017 = histogram_match(coregistered_gs_2017, gs_2016, match_proportion=1.0)

plt.figure(figsize=(20, 15))

plt.subplot(1,3,1)
plt.imshow(gs_2016, cmap='gray')
plt.title('Mosul, Nov 2016')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(gs_2017, cmap='gray')
plt.title('Original 2017')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(equalized_coregistered_gs_2017, cmap='gray')
plt.title('Equalized & Co-registered 2017')
plt.axis('off')


plt.tight_layout()
plt.show()


In [ ]:
difference = equalized_coregistered_gs_2017 - gs_2016
              

basic_change_detection_plot(difference, 
                            title_caption='Method 2:  Difference (2017-2016)', 
                            colorbar_status = 'on',
                            colormap_choice = 'RdBu'
                           )

These results make more sense. The large building, left of center of the image, now shows partial damage which is evident to be true in original images.  The primary roadways in 2017 appear to be covered in dust, making them much brighter than the bare asphalt visible in the previous year. We would expect the difference to be positive, giving them a blue hue.

## Topic 4: Ingesting external data via APIs requiring authorization

Many content providers expose significant amounts of data via APIs and require users to present keys and/or tokens to access them.  I recently was working with <a href="https://spire.com" target="_blank" rel="noopener">Spire</a> <a href="https://en.wikipedia.org/wiki/Automatic_identification_system" target="_blank" rel="noopener">AIS</a> data and found it beneficial to query their APIs directly from a notebook.  The particular query below searches for historical vessel position information based on the footprint and timestamp of a GBDX image and requires specific JSON formatting.

Though Spire is a very useful tool, the API is not readily available in GBDX Notebooks. If you are interested in using the Spire API in your own work, you will need to contact the Spire helpdesk and request and authorization token. You can contact their <a href="https://spire.com/en/customer-resources/developer-portal" target="_blank" rel="noopener">developer support team</a>. If you are interested in a free trial of the Spire Product, you can access it by scrolling down <a href="https://spire.com/en/sample-data" target="_blank" rel="noopener">free trial</a>.

In [ ]:
# connect to gbdx
from gbdxtools import CatalogImage, Interface
gbdx = Interface()

# timestamp manipulation
try:
    import dateutil.parser
except:
    ! pip install python-dateutil
    import dateutil.parser
import datetime
from datetime import timedelta
import time
from time import strftime

# for making API calls
import requests

# for formatting API calls and parsing API responses
import json
    
    
key = 'token_string' # enter Spire API authorization token as a string here, get this from Spire Helpdesk

catalog_id = '1030010080AE6A00' # enter catalog id string of your choosing

strip = CatalogImage(catalog_id, band_type="MS")

strip_box = strip.bounds

print('Image bounds:', strip_box)

strip_min_lon = strip_box[0]
strip_min_lat = strip_box[1]
strip_max_lon = strip_box[2]
strip_max_lat = strip_box[3]

# define a bounding box for the query using 

position = {
    "type": "Polygon",
    "coordinates": [
        [
            [
                strip_min_lon,
                strip_max_lat
            ],
            [
                strip_max_lon,
                strip_max_lat
            ],
            [
                strip_max_lon,
                strip_min_lat
            ],
            [
                strip_min_lon,
                strip_min_lat
            ],
            [
                strip_min_lon,
                strip_max_lat
            ]
          ]
        ]
}

# parse the image's timestamp string convert to a datetime object

strip_timestamp = strip.metadata["image"]["acquisitionDate"]

print('Image timestamp:', strip_timestamp)


t2 = dateutil.parser.parse(strip_timestamp) # window will be centered about the DG image timestamp

#print(t2.strftime('%H:%M:%S'))

spire_time_window_minutes = 30
            
delta_t = datetime.timedelta(minutes= spire_time_window_minutes/2.0)

t1 = t2 - delta_t # beginning of Spire time window

t3 = t2 + delta_t # end of Spire time window

received_after = t1.strftime('%Y-%m-%dT%H:%M:%S')
received_before = t3.strftime('%Y-%m-%dT%H:%M:%S')

r = requests.get('https://ais.spire.com/messages',
                 params={'fields':'decoded',
                         'position':json.dumps(position),
                         'received_after':received_after,
                         'received_before':received_before,
                         'limit':1000},
                 headers={'Authorization': 'Bearer ' + key, 'Accept': 'application/json'})

print()
print('Spire API queried for vessel pings between', received_after,'and',received_before, ',')
print('within the bounding box of', json.dumps(position))


#print(r.url)

if r.status_code == 500:
    print()
    print('Spire API response:  Spire API is temporarily unavailable.')
elif r.status_code == 401:
    print()
    print('Spire API response:  Token is unauthorized.')
elif r.status_code == 200:
    geojson = r.json()
    print()
    print('Spire API response:  Successful Spire API call')

## Conclusion

This is the final notebook in our "Getting Started" tutorial series. 